In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
df_train = pd.read_csv('BBC News Train.csv')
df_test = pd.read_csv('BBC News Test.csv')
df_sample_solution = pd.read_csv('BBC News Sample Solution.csv')